In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import PIL
from keras.applications import VGG16,imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.models import Model
import numpy as np
import glob
import os
from IPython.display import Image,display

import pickle
import time

Using TensorFlow backend.


In [3]:
from PIL import ImageDraw
def draw_bbox(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name']).convert('RGBA')
    
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    rect = PIL.Image.new('RGBA', img.size, (255,255,255,0))
    d = ImageDraw.Draw(img)
    d.rectangle(((lx,ly), (ux,uy)), outline="red")
    return PIL.Image.alpha_composite(img,rect)

def gen_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        display(draw_bbox(image_df.iloc[i]))
        display(draw_bbox(image_df.iloc[match]))

In [4]:
bbox = pd.read_table('Data/DeepFashion/list_bbox.txt',sep='\s+')
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')

tops = bbox.iloc[cat[(cat.category_label>=1) & (cat.category_label<=20)].index]
bottoms = bbox.iloc[cat[(cat.category_label>=21) & (cat.category_label<=36)].index]
dresses = bbox.iloc[cat[(cat.category_label>36)].index]

In [5]:
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

In [7]:
def crop_resize_DF(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name'])
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    
    img = img.crop((lx,ly,ux,uy))
    img = img.resize(inputShape, PIL.Image.ANTIALIAS)
    return img

def preprocess_DF(bbox):
    image = crop_resize_DF(bbox)
    image = img_to_array(image)
    image = np.expand_dims(image,axis=0)
    
    return preprocess(image)

In [9]:
N = 20000
subset = np.random.randint(bbox.shape[0],size=N)
proc_img = []
for i in subset:
    proc_img.append(np.squeeze(preprocess_DF(bbox.iloc[i])))
proc_img = np.array(proc_img)

In [10]:
df = bbox.iloc[subset]

In [11]:
from sys import getsizeof
getsizeof(proc_img)/1e6

12042.240144

In [12]:
import pickle
pickle.dump(bbox.iloc[subset],open('Data/feature_matrix/DF20000_bbox.p','wb'))

In [13]:
base_model = VGG16(weights='imagenet')
base_fc6 = Model(inputs=base_model.input,outputs=base_model.get_layer('fc1').output)
base_fc7 = Model(inputs=base_model.input,outputs=base_model.get_layer('fc2').output)

In [14]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 778865074406547787
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 360448000
locality {
  bus_id: 1
}
incarnation: 17026317379602066617
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0"
]


In [15]:
start = time.time()
fc6_base = base_fc6.predict(proc_img)
print('{:.0f} s'.format(time.time()-start))

AttributeError: 'float' object has no attribute '0f'

In [16]:
fc6_base.shape

(20000, 4096)

In [17]:
start = time.time()
pickle.dump(fc6_base,open('Data/feature_matrix/FC6_base.p','wb'),pickle.HIGHEST_PROTOCOL)
print('{:.0f} s'.format(time.time()-start))

3 s


In [18]:
start = time.time()
fc7_base = base_fc7.predict(proc_img)
pickle.dump(fc7_base,open('Data/feature_matrix/FC7_base.p','wb'))
print('{:.0f} s'.format(time.time()-start))

380 s


In [19]:
from keras.models import load_model
sub_trained = load_model('DF-Subset-Category-trained-7-24.h5')
sub_fc6 = Model(inputs=sub_trained.input,outputs=sub_trained.get_layer('fc1').output)

In [20]:
start = time.time()
fc6_sub = sub_fc6.predict(proc_img)
pickle.dump(fc6_sub,open('Data/feature_matrix/FC6_subset_trained.p','wb'))
print('{:.0f} s'.format(time.time()-start))

378 s


In [21]:
full_trained = load_model('DF-Category-FULL-retrain.h5')
full_fc6 = Model(inputs=full_trained.input,outputs=full_trained.get_layer('fc1').output)

In [22]:
start = time.time()
fc6_full = full_fc6.predict(proc_img)
pickle.dump(fc6_full,open('Data/feature_matrix/FC6_full_trained.p','wb'))
print('{:.0f} s'.format(time.time()-start))

378 s


In [23]:
cat = pd.read_table('Data/DeepFashion/list_category_img.txt',sep='\s+')
df['category_label'] = cat.iloc[df.index]['category_label']

/usr/local/lib/python3.4/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [24]:
from PIL import ImageDraw
def draw_bbox(bbox):
    img = PIL.Image.open('Data/DeepFashion/'+bbox['image_name']).convert('RGBA')
    
    lx = bbox['x_1']
    ly = bbox['y_1']
    ux = bbox['x_2']
    uy = bbox['y_2']
    rect = PIL.Image.new('RGBA', img.size, (255,255,255,0))
    d = ImageDraw.Draw(img)
    d.rectangle(((lx,ly), (ux,uy)), outline="red")
    return PIL.Image.alpha_composite(img,rect)

def gen_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        display(draw_bbox(image_df.iloc[i]))
        display(draw_bbox(image_df.iloc[match]))
        
def test_pairs(N,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    cat_match = []
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j]
        while dist[match]<=0:
            j += 1
            match = np.argsort(dist)[j]

        cat_match.append(image_df.iloc[i]['category_label'] == image_df.iloc[match]['category_label'])
        
    return cat_match

def random_pairs(N,image_df):
    img_index = np.random.randint(image_df.shape[0],size=N)
    match_index = np.random.randint(image_df.shape[0],size=N)
    return (image_df.iloc[img_index]['category_label'].values
            == image_df.iloc[match_index]['category_label'].values)


In [25]:
def find_nearest(dist):
    j=1
    sort = np.argsort(dist)
        
    while dist[sort[j]]<=0:
        j += 1
        
    return sort[j]

def test_pairs_all_D(N,image_df,feature_matrix):
    img_index = np.random.randint(image_df.shape[0],size=N)
    cat_match = {'L2':[],'L1':[],'H':[]}
  
    bin_mat = (feature_matrix > 0)
    
    for i in img_index:
        img_feature = feature_matrix[i,:]
        L2_dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        L1_dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        img_feature = (img_feature>0)
        H_dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        L2_match = find_nearest(L2_dist)
        L1_match = find_nearest(L1_dist)
        H_match = find_nearest(H_dist)

        cat_match['L2'].append(image_df.iloc[i]['category_label'] == image_df.iloc[L2_match]['category_label'])
        cat_match['L1'].append(image_df.iloc[i]['category_label'] == image_df.iloc[L1_match]['category_label'])
        cat_match['H'].append(image_df.iloc[i]['category_label'] == image_df.iloc[H_match]['category_label'])
        
    return cat_match

In [26]:
def print_scores(score):
    print('L1: {:.4f}'.format(np.mean(score['L1'])))
    print('L2: {:.4f}'.format(np.mean(score['L2'])))
    print('H: {:.4f}'.format(np.mean(score['H'])))

In [29]:
r_score = random_pairs(1000,df)
print('P@1: {}'.format(np.mean(r_score)))

P@1: 0.108


In [30]:
fc6_base_score = test_pairs_all_D(1000,df,fc6_base)
print_scores(fc6_base_score)

L1: 0.4650
L2: 0.4640
H: 0.4760


In [31]:
fc7_base_score = test_pairs_all_D(1000,df,fc7_base)
print_scores(fc7_base_score)

L1: 0.4320
L2: 0.4040
H: 0.4520


In [32]:
fc6_sub_score = test_pairs_all_D(1000,df,fc6_sub)
print_scores(fc6_sub_score)

L1: 0.5330
L2: 0.5420
H: 0.5360


In [33]:
fc6_full_score = test_pairs_all_D(1000,df,fc6_full)
print_scores(fc6_full_score)

L1: 0.5670
L2: 0.6000
H: 0.5540


In [46]:
def write_pairs(N,k,image_df,feature_matrix,metric='L2'):
    img_index = np.random.randint(image_df.shape[0],size=N)
    
    if metric == 'H':
        bin_mat = (feature_matrix > 0)
    
    pairs = []
    
    for i in img_index:
        img_feature = feature_matrix[i,:]
        if metric == 'L2':
            dist = np.sum((feature_matrix-img_feature)**2,axis=1)
        elif metric == 'L1':
            dist = np.sum(np.abs(feature_matrix-img_feature),axis=1)
        elif metric == 'H':
            img_feature = (img_feature>0)
            dist = np.sum(np.logical_xor(img_feature,bin_mat),axis=1)
        
        j=1
        match = np.argsort(dist)[j:(j+k)]
        while dist[match[0]]<=0:
            j += 1
            match = np.argsort(dist)[j:(j+k)]
        
        s = image_df.iloc[i]['image_name']
        for m in match:
            s = s + "\t" + image_df.iloc[m]['image_name']
        pairs.append(s)
        
    return pairs

In [56]:
with open('fc6_full_trained_H.txt', 'w') as f:
    f.write('\n'.join(write_pairs(300,3,df,fc6_full,'H')))

In [57]:
with open('fc6_full_trained_L2.txt', 'w') as f:
    f.write('\n'.join(write_pairs(300,3,df,fc6_full,'L2')))

In [58]:
with open('fc6_subset_trained_L2.txt', 'w') as f:
    f.write('\n'.join(write_pairs(300,3,df,fc6_sub,'L2')))

In [59]:
with open('fc6_base_H.txt', 'w') as f:
    f.write('\n'.join(write_pairs(300,3,df,fc6_base,'H')))

In [60]:
with open('fc7_base_H.txt', 'w') as f:
    f.write('\n'.join(write_pairs(300,3,df,fc7_base,'H')))

In [61]:
from sklearn.manifold import TSNE
tsne = TSNE()
fc6_full_embed = tsne.fit_transform(fc6_full)

MemoryError: 

In [62]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

# Plots an image at each x and y location. 
def plotImage(x, y, image, ax):
    im = OffsetImage(load_img(image,target_size=inputShape), zoom=0.15)
    ab = AnnotationBbox(im, (x, y), xycoords='data', frameon=False)
    ax.add_artist(ab)

# Create figure
def plot_tsne(N,embed,image_df,file):
    fig = plt.figure(figsize=(20,20),dpi=300)
    ax = fig.add_subplot(111)

    for i in range(N):
        plotImage(embed[i,0],embed[i,1],
                  'Data/DeepFashion/'+image_df.iloc[i]['image_name'],ax)

    ax.set_ylim(np.min(embed[:N,1]),np.max(embed[:N,1]))
    ax.set_xlim(np.min(embed[:N,0]),np.max(embed[:N,0]))

    plt.savefig(file)

In [63]:
plot_tsne(250,fc6_full_embed,bbox.iloc[subset],'DF_20000_FC6_full_tsne.png')

NameError: name 'fc6_full_embed' is not defined

In [ ]:
fc6_base_embed = tsne.fit_transform(fc6_base)

In [ ]:
plot_tsne(250,fc6_base_embed,bbox.iloc[subset],'DF_20000_FC6_base_tsne.png')